<font size=6>Import

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<font size=6> Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789x '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9',
 11: 'x'}

<font size=6> Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}x{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    ans += ' ' * (DIGITS + 3 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['0x980  ', '72x39  ', '29x942 ', '397x30 ', '4x8    '] ['0     ', '2808  ', '27318 ', '11910 ', '32    ']


<font size=6>Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 3, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 3)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:40000]
train_y = y[:40000]
test_x = x[40000:]
test_y = y[40000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(36000, 7, 12)
(36000, 6, 12)
Validation Data:
(4000, 7, 12)
(4000, 6, 12)
Testing Data:
(40000, 7, 12)
(40000, 6, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False  True False False False
   False]
  [False False False False False False False False False False False
    True]
  [False False  True False False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False False False False False False False False False False
    True]
  [False False  True False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [ True False False False False False Fal

<font size=6> Build Model

In [12]:
print("Build model...")
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 3))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences= True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation= "softmax")))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


<font size=6>Training

In [13]:
for iteration in range(1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=100,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/100
36000/36000 [==============================] - 9s 254us/step - loss: 1.9368 - acc: 0.2898 - val_loss: 1.8327 - val_acc: 0.3142
Epoch 2/100
36000/36000 [==============================] - 8s 214us/step - loss: 1.8107 - acc: 0.3174 - val_loss: 1.7888 - val_acc: 0.3176
Epoch 3/100
36000/36000 [==============================] - 8s 215us/step - loss: 1.7448 - acc: 0.3335 - val_loss: 1.7129 - val_acc: 0.3388
Epoch 4/100
36000/36000 [==============================] - 8s 214us/step - loss: 1.6485 - acc: 0.3644 - val_loss: 1.6009 - val_acc: 0.3785
Epoch 5/100
36000/36000 [==============================] - 8s 214us/step - loss: 1.5402 - acc: 0.3999 - val_loss: 1.4857 - val_acc: 0.4157
Epoch 6/100
36000/36000 [==============================] - 8s 214us/step - loss: 1.4503 - acc: 0.4264 - val_loss: 1.4224 - val_acc: 0.4322
Epoch 7/100
36000/36000 [============================

Epoch 59/100
36000/36000 [==============================] - 8s 210us/step - loss: 0.7122 - acc: 0.7211 - val_loss: 0.7591 - val_acc: 0.6957
Epoch 60/100
36000/36000 [==============================] - 8s 210us/step - loss: 0.7225 - acc: 0.7160 - val_loss: 0.7587 - val_acc: 0.6932
Epoch 61/100
36000/36000 [==============================] - 7s 208us/step - loss: 0.7051 - acc: 0.7261 - val_loss: 0.7441 - val_acc: 0.7003
Epoch 62/100
36000/36000 [==============================] - 8s 212us/step - loss: 0.7111 - acc: 0.7220 - val_loss: 0.7540 - val_acc: 0.6945
Epoch 63/100
36000/36000 [==============================] - 8s 209us/step - loss: 0.6967 - acc: 0.7289 - val_loss: 0.7354 - val_acc: 0.7055
Epoch 64/100
36000/36000 [==============================] - 8s 210us/step - loss: 0.7019 - acc: 0.7253 - val_loss: 0.7501 - val_acc: 0.6985
Epoch 65/100
36000/36000 [==============================] - 8s 211us/step - loss: 0.7040 - acc: 0.7250 - val_loss: 0.7871 - val_acc: 0.6835
Epoch 66/100
36000/3

<font size=6>Testing

In [14]:
print("MSG : Prediction")


MSG : Prediction
